# List of Packages to Download


In [ ]:
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub

In [ ]:
#!pip install --upgrade gpt4all

In [ ]:
pip install langchain-huggingface

In [3]:
!pip install --upgrade --quiet  langchain sentence_transformers

# 1. Get a Data Loader


In [5]:
from langchain_community.document_loaders import WebBaseLoader


In [ ]:
loader = WebBaseLoader("https://www.foxsports.com/nba/lebron-james-player-game-log?season=2023&seasonType=reg")
data = loader.load()
data

# 2. Convert data to Vector Database


In [7]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings



In [ ]:
embeddings_model = HuggingFaceEmbeddings() #sentence-transformers/all-mpnet-base-v2

In [9]:
Chroma.from_documents(documents=data, embedding=embeddings_model, persist_directory="./chroma_db")

# 3. Make a RAG pipeline


In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [ ]:
model_id = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
hf = HuggingFacePipeline(pipeline=pipe)

In [12]:
llm = hf

#uncomment if using ollama
#llm = ChatOllama(model="qwen2")

prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=HuggingFaceEmbeddings())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [14]:
question = "Who's stats are these"
result = qa_chain({"query": question })

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
result["result"]